# ML Zoomcamp 2023 - Homework #5

In this homework, we will use Bank credit scoring dataset from
Question 1

    Install Pipenv
    What's the version of pipenv you installed?
    Use --version to find out

In [2]:
!pipenv --version

pipenv, version 2023.10.3


Question 2

    Use Pipenv to install Scikit-Learn version 1.3.1
    What's the first hash for scikit-learn you get in Pipfile.lock?

"sha256:0c275a06c5190c5ce00af0acbb61c06374087949f643ef32d355ece12c4db043"

Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['job','duration', 'poutcome']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)

And then saved with Pickle. Download them:

    DictVectorizer
    LogisticRegression

With wget:

PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin

In [3]:
PREFIX="https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework"
!wget "$PREFIX/model1.bin"
!wget "$PREFIX/dv.bin"


--2023-10-14 12:11:06--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... Read error (Decryption has failed.) in headers.
Retrying.

--2023-10-14 12:11:07--  (try: 2)  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/model1.bin
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 842 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     842  --.-KB/s    in 0s      

2023-10-14 12:11:08 (37.9 MB/s) - ‘model1.bin’ saved [842/842]

-

In [4]:
!ls

 dv.bin  'HW5_Solution(1).ipynb'   model1.bin   Untitled.ipynb


Question 3

Let's use these models!

    Write a script for loading these models with pickle
    Score this client:

{"job": "retired", "duration": 445, "poutcome": "success"}

What's the probability that this client will get a credit?

    0.162
    0.392
    0.652
    0.902

In [5]:
import pickle

from flask import Flask
from flask import request
from flask import jsonify

import requests

If you're getting errors when unpickling the files, check their checksum:

In [6]:
!md5sum model1.bin dv.bin

8ebfdf20010cfc7f545c43e3b52fc8a1  model1.bin
924b496a89148b422c74a62dbc92a4fb  dv.bin


In [11]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

model_file = 'model1.bin'
dv_file = 'dv.bin'

with open(model_file, 'rb') as f_in:
    model1 = pickle.load(f_in)
    f_in.close()
    
with open(dv_file, 'rb') as f_in:
    dv1 = pickle.load(f_in)
    f_in.close()


client = {"job": "retired", "duration": 445, "poutcome": "success"}

X = dv1.transform([client])
y_pred = model1.predict_proba(X)[0, 1]
print('the probability that this client will get a credit {}'.format(y_pred))

the probability that this client will get a credit 0.9019309332297606


Question 4

Now let's serve this model as a web service

    Install Flask and gunicorn (or waitress, if you're on Windows)
    Write Flask code for serving the model
    Now score this client using requests:

import pickle

from flask import Flask
from flask import request
from flask import jsonify

def load(filename):
    with open(filename, 'rb') as file_in:
        return pickle.load(file_in)

dv = load('dv.bin')
model = load('model1.bin')

app = Flask('credit')

@app.route('/predict', methods=['POST'])
def predict():
    client = request.get_json()

    X = dv.transform(client)
    y_pred = model.predict_proba(X)[0, 1]
    get_credit = y_pred >= 0.5

    result = {
        'get_credit_prob': float(y_pred),
        'get_credit': bool(get_credit),

        }

    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

In [6]:
import requests

# Define the URL where your Flask service is running
url = "http://localhost:9696/predict"  # Adjust the URL as needed

# Define the client data as a dictionary
client = {
    "job": "unknown",
    "duration": 270,
    "poutcome": "failure"
}
result = requests.post(url, json=client).json()

print("The probability that this client will get a credit? {}".format(result['get_credit_prob']))

The probability that this client will get a credit? 0.13968947052356817


Docker
Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.10.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.10.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

FROM python:3.10.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]

We already built it and then pushed it to svizor/zoomcamp-model:3.10.12-slim.

# pulling the image 
┌──[~/machine-learning-zoomcamp-projects]
└─$ sudo docker pull svizor/zoomcamp-model:3.10.12-slim
[sudo] password for edwin: 
3.10.12-slim: Pulling from svizor/zoomcamp-model
52d2b7f179e3: Pull complete 
2b8a9a2240c1: Pull complete 
618a49bbc6c6: Pull complete 
708d13076071: Pull complete 
90b76fc2ebde: Pull complete 
a7c599e7b037: Pull complete 
faa7e3220190: Pull complete 
Digest: sha256:e8441100b9d8da56344f50c673eb2daded3c61ce9565e45c3592c02f34fb3149
Status: Downloaded newer image for svizor/zoomcamp-model:3.10.12-slim
docker.io/svizor/zoomcamp-model:3.10.12-slim

Question 5

Download the base image svizor/zoomcamp-model:3.10.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

    47 MB
    147 MB
    374 MB
    574 MB

# Building the image 
┌──-[~/machine-learning-zoomcamp-projects/homework_5]
└─$ sudo docker build -t credit_score .                                             
Sending build context to Docker daemon    151kB
Step 1/10 : FROM svizor/zoomcamp-model:3.10.12-slim
 ---> 08266c8f0c4b
Step 2/10 : RUN pip install pipenv
 ---> Using cache
 ---> 133f868cdf67
Step 3/10 : RUN pip install Flask
 ---> Using cache
 ---> 8cd33f81def7
Step 4/10 : COPY ["Pipfile", "Pipfile.lock", "./"]
 ---> Using cache
 ---> 7aa4d0e58de8
Step 5/10 : RUN pipenv install --system --deploy
 ---> Using cache
 ---> 97082b822f96
Step 6/10 : COPY ["dv.bin", "/"]
 ---> Using cache
 ---> c16d8833c1e5
Step 7/10 : COPY ["model1.bin", "/model1.bin"]
 ---> Using cache
 ---> 6be1591e425a
Step 8/10 : COPY ["quiz_6.py", "./"]
 ---> 66498d74a2d9
Step 9/10 : EXPOSE 9616
 ---> Running in 34adf9af9ccf
Removing intermediate container 34adf9af9ccf
 ---> 0bb174865510
Step 10/10 : ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9616", "quiz_6:app"]
 ---> Running in e439a956f7bc
Removing intermediate container e439a956f7bc
 ---> 062a8e29d18f
Successfully built 062a8e29d18f
Successfully tagged credit_score:latest
                                                                                              
└─$ sudo docker run -it --rm -p 1616:1616 --name credit_score_container credit_score

In [10]:
import requests

url = 'http://localhost:9616/predict'

client = {"job": "retired", "duration": 445, "poutcome": "success"}

result = requests.post(url, json=client).json()

print(result)

{'get_credit': True, 'get_credit_prob': 0.9019309332297606}
